In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import emoji

In [2]:
df = pd.read_excel('labeled data.xlsx')

In [3]:
print(f'Строк: {df.shape[0]:,}, столбцов: {df.shape[1]}')

Строк: 2,373, столбцов: 8


In [4]:
df.head()

,id,rating,comment,Нравится скорость отработки заявок,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Понравилось выполнение заявки,Вопрос решен
0,2945792,5,спасибо,NaN,NaN,NaN,1.0,NaN
1,3234340,5,спасибо!,NaN,NaN,NaN,1.0,NaN
2,3380332,5,Отлично,NaN,NaN,NaN,1.0,NaN
3,3381812,5,Благодарю за оперативное решение проблемы !,1.0,NaN,NaN,NaN,NaN
4,3461991,5,Прекрасный специалист! Побольше таких,NaN,NaN,1.0,NaN,NaN


In [5]:
label_cols = [c for c in df.columns if c not in ('id', 'rating', 'comment')]

label_counts = df[label_cols].notna().sum().sort_values(ascending=False)

all_zero_count = (df[label_cols].fillna(0).sum(axis=1) == 0).sum()

label_counts["Все классы 0"] = all_zero_count
print(label_counts)

Понравилось выполнение заявки           1048
Нравится качество выполнения заявки     1044
Нравится скорость отработки заявок       790
Вопрос решен                             768
Нравится качество работы сотрудников     606
Все классы 0                             183
dtype: int64


In [6]:
duplicates_comments = df[df['comment'].duplicated(keep=False)]['comment']

mask_short = df['comment'].str.len() < 10
short_comments = df.loc[mask_short, ['comment']].copy()
short_comments['length'] = short_comments['comment'].str.len()

duplicates_count = duplicates_comments.shape[0]
short_count = short_comments.shape[0]

print(f"Всего дубликатов отзывов: {duplicates_count}")
print("Примеры дубликатов:")
print(duplicates_comments.head(10).to_string(index=False))

print(f"Всего коротких отзывов: {short_count}")
print("Примеры коротких отзывов:")
print(short_comments.head(10).to_string(index=False))

Всего дубликатов отзывов: 702
Примеры дубликатов:
                                     спасибо
                                    спасибо!
                                    Отлично 
Благодарю за оперативное решение проблемы ! 
       Прекрасный специалист! Побольше таких
                                    Спасибо 
                     пересчет, и скорость \n
         Спасибо за обновление покраски стен
                            выполнили быстро
                                   Спасибо! 
Всего коротких отзывов: 444
Примеры коротких отзывов:
   comment  length
   спасибо       7
  спасибо!       8
  Отлично        8
  Спасибо        8
 Спасибо!        9
спасибо \n       9
  Спасибо        8
   спасибо       7
  Спасибо!       8
      Норм       4


In [7]:
df_no_duplicates = df[~df['comment'].duplicated(keep=False)].copy()

df_cleaned = df_no_duplicates[df_no_duplicates['comment'].str.len() >= 10].copy()

print(f"Исходно было строк: {len(df)}")
print(f"После удаления дубликатов: {len(df_no_duplicates)}")
print(f"После удаления коротких (<10): {len(df_cleaned)}")

Исходно было строк: 2373
После удаления дубликатов: 1671
После удаления коротких (<10): 1611


In [8]:
def replace_emoji_to_text(text):
    return ''.join(
        emoji.demojize(char, delimiters=(" :", ": ")) if char in emoji.EMOJI_DATA else char
        for char in text
    )
df_cleaned['comment'] = df_cleaned['comment'].apply(replace_emoji_to_text)

In [9]:
label_cols = [c for c in df_cleaned.columns if c not in ('id', 'rating', 'comment')]

label_counts = df_cleaned[label_cols].notna().sum().sort_values(ascending=False)

all_zero_count = (df_cleaned[label_cols].fillna(0).sum(axis=1) == 0).sum()

label_counts["Все классы = 0"] = all_zero_count

print(label_counts)

Нравится качество выполнения заявки     748
Нравится скорость отработки заявок      633
Вопрос решен                            552
Нравится качество работы сотрудников    537
Понравилось выполнение заявки           521
Все классы = 0                          178
dtype: int64


In [10]:
output_path = 'cleaned_dataset.xlsx'
df_cleaned.to_excel(output_path, index=False)

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
from sklearn.utils import resample
from skmultilearn.model_selection import iterative_train_test_split

np.random.seed(42)
df = (
    pd.read_excel("cleaned_dataset.xlsx")
      .sample(frac=1, random_state=42)
      .reset_index(drop=True)
)

label_cols = [c for c in df.columns if c not in ["id", "rating", "comment"]]
X = df["comment"].astype(str).values
Y = df[label_cols].fillna(0).astype(int).values

X_tr, y_tr, X_te, y_te = iterative_train_test_split(X.reshape(-1,1), Y, test_size = 0.2)
                                                    
                                                    
X_tr = X_tr.ravel()
X_te = X_te.ravel()

train_df = pd.DataFrame({"comment": X_tr})
train_df[label_cols] = y_tr
test_df  = pd.DataFrame({"comment": X_te})
test_df[label_cols]  = y_te

train_df.to_excel("train_80_old.xlsx", index=False)
test_df.to_excel("test_20_old.xlsx", index=False)

print("Train shape:", train_df.shape)
print(train_df[label_cols].sum())
print("Test shape:", test_df.shape)
print(test_df[label_cols].sum())

/Users/sabinakalbieva/Documents/add_hse/tools_ad/venv_rubert_sentiment_clean/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sabinakalbieva/Documents/add_hse/tools_ad/venv_rubert_sentiment_clean/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train shape: (1288, 6)
Нравится скорость отработки заявок      506
Нравится качество выполнения заявки     598
Нравится качество работы сотрудников    430
Понравилось выполнение заявки           417
Вопрос решен                            442
dtype: int64
Test shape: (323, 6)
Нравится скорость отработки заявок      127
Нравится качество выполнения заявки     150
Нравится качество работы сотрудников    107
Понравилось выполнение заявки           104
Вопрос решен                            110
dtype: int64


In [12]:
print("\nКодируем LaBSE-эмбеддинги …")
labse  = SentenceTransformer("sentence-transformers/LaBSE")
emb_tr = labse.encode(train_df["comment"].tolist(), show_progress_bar=True)
emb_te = labse.encode(test_df["comment"].tolist(),  show_progress_bar=True)

print("Обучаем KNN-5 (One-vs-Rest) …")
clf    = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5))
clf.fit(emb_tr, train_df[label_cols].values)

y_pred   = clf.predict(emb_te)
macro_f1 = f1_score(test_df[label_cols].values, y_pred, average="macro")
print(f"\nBaseline macro-F1 = {macro_f1:.3f}")


Кодируем LaBSE-эмбеддинги …


/Users/sabinakalbieva/Documents/add_hse/tools_ad/venv_rubert_sentiment_clean/lib/python3.9/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 11/11 [00:01<00:00,  8.51it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Обучаем KNN-5 (One-vs-Rest) …

Baseline macro-F1 = 0.672


### ЧАТ ГПТ 1

In [13]:
import random
from random import sample

random.seed(42)

var = train_df.groupby(['Нравится скорость отработки заявок',
       'Нравится качество выполнения заявки',
       'Нравится качество работы сотрудников', 'Понравилось выполнение заявки',
       'Вопрос решен'])['comment'].apply(list)
d = {index: sample(var[index], min(len(var[index]), 5)) for index in var.index}

In [14]:
d = {
    key: '\n'.join(f'"{s}"' for s in value) 
    for key, value in d.items()
}

In [15]:
d

{(0,
  0,
  0,
  0,
  0): '"причину выяснили, но дверь как была открыта, так и осталась. вопрос не был решен."\n"так и не работает дверь входная и дамовон"\n"со мной никто не связался и никто не снял показания"\n"мария доброе утро!  абсолютно ничего на этаже не отмыто, химией не работали это видно на фото, так же отсутствует запах. прошлый раз когда администратор действительно отмывала был запах химии и плитка была хоть и в небольших белых пятнах но без разводов. сейчас же такое ощущение что снова помыли полы грязной тряпкой и водой! прошу всетаки разобраться в сложившейся ситуации."\n"жду ответа"',
 (0,
  0,
  0,
  0,
  1): '"можно было б и быстрее"\n"Претензий нет"\n"спасибо за отмену заявки по моей просьбе"\n"обычная услуга. Цена, правда, завышена (за около 10ти минут работы). "\n"еще бы плитку восстановили, пока никто не сломал ноги"',
 (0,
  0,
  0,
  1,
  0): '"в лучшем виде "\n"спасибо, что решили свою проблему, а то как то не очень, платишь вовремя, а тебе про долг пишут. "\n"П

In [ ]:
import re, time, random
import pandas as pd
from tqdm import tqdm
from typing import Optional
from openai import OpenAI, APIError, APIConnectionError, RateLimitError

OPENAI_KEY   = "sk-proj-"
client = OpenAI(api_key=OPENAI_KEY)

train = pd.read_excel("train_80_old.xlsx")

SYSTEM = ("Ты пишешь короткие (1–2-3 предложения) отзывы жильцов "
          "управляющей компании. Пиши строго на русском.")

PROMPT_BASE = (
    "Примеры отзывoв:\n«{ex}»\n\n"
    "{emoji_hint}"
    "Напиши 3-10 похожих, но уникальных отзывов (1–2-3-4 предложения)."
    "Количество отзывов определяй исходя из кол-ва примеров."
    "Не копируй примеры. Отделяй каждый отзыв новой строкой"
)

EMOJI_RE = re.compile(r":[a-z0-9_]+:", flags=re.I)

def gpt_review(example):
    has_code = bool(EMOJI_RE.search(example))
    
    emoji_hint = ("Если в примере есть код эмодзи вида :smile:, "
                  "то сохрани стиль — вставь 1 похожий код эмодзи.\n\n"
                  ) if has_code else ""
    
    prompt = PROMPT_BASE.format(ex=example, emoji_hint=emoji_hint)

    for _ in range(3):
        try:
            r = client.chat.completions.create(
                model="gpt-3.5-turbo-0125",
                temperature=0.7,
                messages=[
                    {"role": "system", "content": SYSTEM},
                    {"role": "user",   "content": prompt}
                ]
            )
            return r.choices[0].message.content.strip()
        except (APIError, APIConnectionError, RateLimitError):
            time.sleep(2)
            print(f'ERROR - {example}')
    return None

synthetic1 = {}
for key, value in d.items():
    example = value
    print(f"{key}: генерируем для {value}")
    
    txt = gpt_review(example)
    if txt:
        synthetic1[key] = txt

(0, 0, 0, 0, 0): генерируем для "причину выяснили, но дверь как была открыта, так и осталась. вопрос не был решен."
"так и не работает дверь входная и дамовон"
"со мной никто не связался и никто не снял показания"
"мария доброе утро!  абсолютно ничего на этаже не отмыто, химией не работали это видно на фото, так же отсутствует запах. прошлый раз когда администратор действительно отмывала был запах химии и плитка была хоть и в небольших белых пятнах но без разводов. сейчас же такое ощущение что снова помыли полы грязной тряпкой и водой! прошу всетаки разобраться в сложившейся ситуации."
"жду ответа"
(0, 0, 0, 0, 1): генерируем для "можно было б и быстрее"
"Претензий нет"
"спасибо за отмену заявки по моей просьбе"
"обычная услуга. Цена, правда, завышена (за около 10ти минут работы). "
"еще бы плитку восстановили, пока никто не сломал ноги"
(0, 0, 0, 1, 0): генерируем для "в лучшем виде "
"спасибо, что решили свою проблему, а то как то не очень, платишь вовремя, а тебе про долг пишут. "
"

In [17]:
save1 = pd.DataFrame.from_dict(synthetic1, orient='index', columns=['Текст'])
save1.to_excel("chatgpt1_old.xlsx")

In [18]:
import re
import ast
import pandas as pd

FILE_IN  = "chatgpt1_old.xlsx"
FILE_OUT = "parsed_comments1_old.xlsx" 

QUOTE_RE = re.compile(
    r'«([^»]+)»'
    r'|“([^”]+)”' 
    r'|"([^"]+)"', 
    flags=re.S,
)

def explode_comments(df_raw: pd.DataFrame) -> pd.DataFrame:
    """Разворачивает ячейки с кучей комментариев в «плоский» DataFrame."""
    rows = []

    for _, row in df_raw.iterrows():
        labels = ast.literal_eval(row.iloc[0])
        text   = str(row.iloc[1]).strip()

        parts = []
        for m in QUOTE_RE.finditer(text):
            part = next(g for g in m.groups() if g)
            parts.append(part.strip())

        if not parts:
            for line in re.split(r'[\r\n]+', text):
                line = line.strip()
                if not line:
                    continue
                line = re.sub(r'^[-–•\s]*', '', line)
                line = line.strip('«»" ')
                parts.append(line)

        for c in parts:
            rows.append({
                "comment": c,
                "Нравится скорость отработки заявок": labels[0],
                "Нравится качество выполнения заявки": labels[1],
                "Нравится качество работы сотрудников": labels[2],
                "Понравилось выполнение заявки": labels[3],
                "Вопрос решен": labels[4],
            })

    return pd.DataFrame(rows)

if __name__ == "__main__":
    df_raw    = pd.read_excel(FILE_IN)
    df_parsed1 = explode_comments(df_raw)

    df_parsed1.to_excel(FILE_OUT, index=False)
    print(df_parsed1.head())

                                             comment  \
0  Не устранили протечку в ванной, хотя обещали с...   
1  Мусорные баки на территории двора не убираются...   
2  Очень долго реагируют на заявки по ремонту в к...   
3  Не проводятся регулярные дезинфекции в подъезд...   
4  Касса управляющей компании работает только в б...   

   Нравится скорость отработки заявок  Нравится качество выполнения заявки  \
0                                   0                                    0   
1                                   0                                    0   
2                                   0                                    0   
3                                   0                                    0   
4                                   0                                    0   

   Нравится качество работы сотрудников  Понравилось выполнение заявки  \
0                                     0                              0   
1                                     0       

In [19]:
df_full1 = pd.concat([df_parsed1, train], ignore_index=True)

In [20]:
print(f'Строк: {df_full1.shape[0]:,}, столбцов: {df_full1.shape[1]}')

Строк: 1,455, столбцов: 6


In [21]:
print("\nКодируем LaBSE-эмбеддинги …")
labse  = SentenceTransformer("sentence-transformers/LaBSE")
emb_tr = labse.encode(df_full1["comment"].tolist(), show_progress_bar=True)
emb_te = labse.encode(test_df["comment"].tolist(),  show_progress_bar=True)

print("Обучаем KNN-5 (One-vs-Rest) …")
clf    = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5))
clf.fit(emb_tr, df_full1[label_cols].values)

y_pred   = clf.predict(emb_te)
macro_f1 = f1_score(test_df[label_cols].values, y_pred, average="macro")
print(f"\nBaseline macro-F1 = {macro_f1:.3f}")


Кодируем LaBSE-эмбеддинги …


/Users/sabinakalbieva/Documents/add_hse/tools_ad/venv_rubert_sentiment_clean/lib/python3.9/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 11/11 [00:00<00:00, 13.20it/s]


Обучаем KNN-5 (One-vs-Rest) …

Baseline macro-F1 = 0.674


### ЧАТ ГПТ 2

In [22]:
import random
from random import sample

random.seed(42)

var = train_df.groupby(['Нравится скорость отработки заявок',
       'Нравится качество выполнения заявки',
       'Нравится качество работы сотрудников', 'Понравилось выполнение заявки',
       'Вопрос решен'])['comment'].apply(list)
d = {index: var[index] for index in var.index}

In [23]:
d = {
    key: '\n'.join(f'"{s}"' for s in value) 
    for key, value in d.items()
}

In [ ]:
import re, time, random
import pandas as pd
from tqdm import tqdm
from typing import Optional
from openai import OpenAI, APIError, APIConnectionError, RateLimitError

OPENAI_KEY   = "sk-proj-"
client = OpenAI(api_key=OPENAI_KEY)

train = pd.read_excel("train_80_old.xlsx")

SYSTEM = ("Ты пишешь короткие (1–2-3 предложения) отзывы жильцов "
          "управляющей компании. Пиши строго на русском.")

PROMPT_BASE = (
    "Примеры отзывoв:\n«{ex}»\n\n"
    "{emoji_hint}"
    "Напиши столько же отзывов, сколько примеров. Отзывы должны быть уникальны (1-6 предложений)."
    "Не копируй примеры. Отделяй каждый отзыв новой строкой"
)

EMOJI_RE = re.compile(r":[a-z0-9_]+:", flags=re.I)

def gpt_review(example):
    has_code = bool(EMOJI_RE.search(example))
    
    emoji_hint = ("Если в примере есть код эмодзи вида :smile:, "
                  "то сохрани стиль — вставь 1 похожий код эмодзи.\n\n"
                  ) if has_code else ""
    
    prompt = PROMPT_BASE.format(ex=example, emoji_hint=emoji_hint)

    for _ in range(3):
        try:
            r = client.chat.completions.create(
                model="gpt-3.5-turbo-0125",
                temperature=0.7,
                messages=[
                    {"role": "system", "content": SYSTEM},
                    {"role": "user", "content": prompt}
                ]
            )
            return r.choices[0].message.content.strip()
        except (APIError, APIConnectionError, RateLimitError):
            time.sleep(2)
            print(f'ERROR - {example}')
    return None

synthetic2 = {}
for key, value in d.items():
    example = value
    print(f"{key}: генерируем для {value}")
    
    txt = gpt_review(example)
    if txt:
        synthetic2[key] = txt

(0, 0, 0, 0, 0): генерируем для "чего не сделано"
"управляющий железняков так и не сделал мне перерасчет"
"жду перерасчета в квитанции, получается ?? тогда пока не оплачиваю тут выставленный счет"
"не понимаю как можно отрапортовать что работа выполнена , когда как скрипела дверь и хлопала так и все осталось без изменений."
"пожалуйста свяжитесь со мной"
"до сих пор в приложении нет актуальной информации о поверке счётчиков, я не могу из за этого передать показания."
"так и не работает дверь входная и дамовон"
"не отображаются счета на оплату"
"приходила в ук мне сказали за деньги купить себе новый ключ, на каком основании? выдали не рабочие и теперь платя 2500 за домофон я ещё и ключи должна покупать, соседям дали бесплатно"
"резиновые прокладки в окнах заменены частично. от продувания не помогло"
"проблема не устранена!!!!"
"вы не ответили, внесли ли вы изменения. меня волнует начисления с января 2025 года. я вам предоставила адресную справку которая от 03.01.25 года, в которой говор

In [25]:
save2 = pd.DataFrame.from_dict(synthetic2, orient='index', columns=['Текст'])
save2.to_excel("chatgpt2_old.xlsx")

In [26]:
import re
import ast
import pandas as pd

FILE_IN  = "chatgpt2_old.xlsx"
FILE_OUT = "parsed_comments2_old.xlsx"

QUOTE_RE = re.compile(
    r'«([^»]+)»'
    r'|“([^”]+)”'
    r'|"([^"]+)"', 
    flags=re.S,
)

def explode_comments(df_raw: pd.DataFrame) -> pd.DataFrame:
    """Разворачивает ячейки с кучей комментариев в «плоский» DataFrame."""
    rows = []

    for _, row in df_raw.iterrows():
        labels = ast.literal_eval(row.iloc[0])
        text   = str(row.iloc[1]).strip()

        parts = []
        for m in QUOTE_RE.finditer(text):
            part = next(g for g in m.groups() if g)
            parts.append(part.strip())

        if not parts:
            for line in re.split(r'[\r\n]+', text):
                line = line.strip()
                if not line:
                    continue
                line = re.sub(r'^[-–•\s]*', '', line)
                line = line.strip('«»" ')
                parts.append(line)

        for c in parts:
            rows.append({
                "comment": c,
                "Нравится скорость отработки заявок": labels[0],
                "Нравится качество выполнения заявки": labels[1],
                "Нравится качество работы сотрудников": labels[2],
                "Понравилось выполнение заявки": labels[3],
                "Вопрос решен": labels[4],
            })

    return pd.DataFrame(rows)

if __name__ == "__main__":
    df_raw    = pd.read_excel(FILE_IN)
    df_parsed2 = explode_comments(df_raw)

    df_parsed2.to_excel(FILE_OUT, index=False)
    print(df_parsed2.head())

                                             comment  \
0  Нет реакции на мои заявки уже неделю. Не понят...   
1  Управляющая компания не выполняет свои обязанн...   
2  Просила сделать перерасчет за отопление, так и...   
3  Дверь в подъезде не закрывается, уже неделю ни...   
4  Счет за коммунальные услуги пришел с огромным ...   

   Нравится скорость отработки заявок  Нравится качество выполнения заявки  \
0                                   0                                    0   
1                                   0                                    0   
2                                   0                                    0   
3                                   0                                    0   
4                                   0                                    0   

   Нравится качество работы сотрудников  Понравилось выполнение заявки  \
0                                     0                              0   
1                                     0       

In [27]:
df_full2 = pd.concat([df_parsed2, train], ignore_index=True)

In [28]:
print(f'Строк: {df_full2.shape[0]:,}, столбцов: {df_full2.shape[1]}')

Строк: 1,466, столбцов: 6


In [29]:
print("\nКодируем LaBSE-эмбеддинги …")
labse  = SentenceTransformer("sentence-transformers/LaBSE")
emb_tr = labse.encode(df_full2["comment"].tolist(), show_progress_bar=True)
emb_te = labse.encode(test_df["comment"].tolist(),  show_progress_bar=True)

print("Обучаем KNN-5 (One-vs-Rest) …")
clf    = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5))
clf.fit(emb_tr, df_full2[label_cols].values)

y_pred   = clf.predict(emb_te)
macro_f1 = f1_score(test_df[label_cols].values, y_pred, average="macro")
print(f"\nBaseline macro-F1 = {macro_f1:.3f}")


Кодируем LaBSE-эмбеддинги …


/Users/sabinakalbieva/Documents/add_hse/tools_ad/venv_rubert_sentiment_clean/lib/python3.9/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 11/11 [00:00<00:00, 12.82it/s]


Обучаем KNN-5 (One-vs-Rest) …

Baseline macro-F1 = 0.672


### ЧАТ ГПТ 3

In [30]:
import random
from random import sample

random.seed(42)

var = train_df.groupby(['Нравится скорость отработки заявок',
       'Нравится качество выполнения заявки',
       'Нравится качество работы сотрудников', 'Понравилось выполнение заявки',
       'Вопрос решен'])['comment'].apply(list)
d = {index: var[index] for index in var.index}

In [31]:
d = {
    key: '\n'.join(f'"{s}"' for s in value) 
    for key, value in d.items()
}

In [ ]:
import re, time, random
import pandas as pd
from tqdm import tqdm
from typing import Optional
from openai import OpenAI, APIError, APIConnectionError, RateLimitError

OPENAI_KEY   = "sk-proj-"
client = OpenAI(api_key=OPENAI_KEY)

train = pd.read_excel("train_80_old.xlsx")

SYSTEM = ("Ты пишешь короткие отзывы жильцов "
          "управляющей компании. Пиши строго на русском.")

PROMPT_BASE = (
    "Примеры отзывoв:\n«{ex}»\n\n"
    "{emoji_hint}"
    "Напиши столько {n} похожих отзывов. Отзывы должны быть уникальны (1-6 предложений)."
    "Не копируй примеры. Отделяй каждый отзыв новой строкой"
)

EMOJI_RE = re.compile(r":[a-z0-9_]+:", flags=re.I)

def gpt_review(example, number):
    has_code = bool(EMOJI_RE.search(example))
    
    emoji_hint = ("Если в примере есть код эмодзи вида :smile:, "
                  "то сохрани стиль — вставь 1 похожий код эмодзи.\n\n"
                  ) if has_code else ""
    
    prompt = PROMPT_BASE.format(ex=example, n=number, emoji_hint=emoji_hint)

    for _ in range(3):
        try:
            r = client.chat.completions.create(
                model="gpt-3.5-turbo-0125",
                temperature=0.7,
                messages=[
                    {"role": "system", "content": SYSTEM},
                    {"role": "user",   "content": prompt}
                ]
            )
            return r.choices[0].message.content.strip()
        except (APIError, APIConnectionError, RateLimitError):
            time.sleep(2)
            print(f'ERROR - {example}')
    return None

synthetic3 = {}
for key, value in d.items():
    example = value
    print(f"{key}: генерируем для {value}")
    
    txt = gpt_review(example, example.count('\n')+1)
    if txt:
        synthetic3[key] = txt

(0, 0, 0, 0, 0): генерируем для "чего не сделано"
"управляющий железняков так и не сделал мне перерасчет"
"жду перерасчета в квитанции, получается ?? тогда пока не оплачиваю тут выставленный счет"
"не понимаю как можно отрапортовать что работа выполнена , когда как скрипела дверь и хлопала так и все осталось без изменений."
"пожалуйста свяжитесь со мной"
"до сих пор в приложении нет актуальной информации о поверке счётчиков, я не могу из за этого передать показания."
"так и не работает дверь входная и дамовон"
"не отображаются счета на оплату"
"приходила в ук мне сказали за деньги купить себе новый ключ, на каком основании? выдали не рабочие и теперь платя 2500 за домофон я ещё и ключи должна покупать, соседям дали бесплатно"
"резиновые прокладки в окнах заменены частично. от продувания не помогло"
"проблема не устранена!!!!"
"вы не ответили, внесли ли вы изменения. меня волнует начисления с января 2025 года. я вам предоставила адресную справку которая от 03.01.25 года, в которой говор

In [33]:
save3 = pd.DataFrame.from_dict(synthetic3, orient='index', columns=['Текст'])
save3.to_excel("chatgpt3_old.xlsx")

In [34]:
import re
import ast
import pandas as pd

FILE_IN  = "chatgpt3_old.xlsx" 
FILE_OUT = "parsed_comments3_old.xlsx" 

QUOTE_RE = re.compile(
    r'«([^»]+)»'
    r'|“([^”]+)”'
    r'|"([^"]+)"',
    flags=re.S,
)

def explode_comments(df_raw: pd.DataFrame) -> pd.DataFrame:
    """Разворачивает ячейки с кучей комментариев в «плоский» DataFrame."""
    rows = []

    for _, row in df_raw.iterrows():
        # 1. извлекаем кортеж меток
        labels = ast.literal_eval(row.iloc[0])
        text   = str(row.iloc[1]).strip()

        parts = []
        for m in QUOTE_RE.finditer(text):
            part = next(g for g in m.groups() if g) 
            parts.append(part.strip())

        if not parts:
            for line in re.split(r'[\r\n]+', text):
                line = line.strip()
                if not line:
                    continue
                line = re.sub(r'^[-–•\s]*', '', line)
                line = line.strip('«»" ')
                parts.append(line)

        for c in parts:
            rows.append({
                "comment": c,
                "Нравится скорость отработки заявок":labels[0],
                "Нравится качество выполнения заявки": labels[1],
                "Нравится качество работы сотрудников": labels[2],
                "Понравилось выполнение заявки": labels[3],
                "Вопрос решен": labels[4],
            })

    return pd.DataFrame(rows)

if __name__ == "__main__":
    df_raw    = pd.read_excel(FILE_IN)
    df_parsed3 = explode_comments(df_raw)

    df_parsed3.to_excel(FILE_OUT, index=False)
    print(df_parsed3.head())

                                             comment  \
0  На сколько надо ждать ответа на вопрос? Уже не...   
1  Здесь никакой реакции на заявку, похоже, пробл...   
2  Просила убрать демонтажное окно из подъезда, а...   
3  Не понимаю, почему нельзя сразу дать четкий от...   
4  Пока не создам заявку, ничего не делают, тольк...   

   Нравится скорость отработки заявок  Нравится качество выполнения заявки  \
0                                   0                                    0   
1                                   0                                    0   
2                                   0                                    0   
3                                   0                                    0   
4                                   0                                    0   

   Нравится качество работы сотрудников  Понравилось выполнение заявки  \
0                                     0                              0   
1                                     0       

In [35]:
df_full3 = pd.concat([df_parsed3, train], ignore_index=True)

In [36]:
print(f'Строк: {df_full3.shape[0]:,}, столбцов: {df_full3.shape[1]}')

Строк: 2,227, столбцов: 6


In [37]:
print("\nКодируем LaBSE-эмбеддинги …")
labse  = SentenceTransformer("sentence-transformers/LaBSE")
emb_tr = labse.encode(df_full3["comment"].tolist(), show_progress_bar=True)
emb_te = labse.encode(test_df["comment"].tolist(),  show_progress_bar=True)

print("Обучаем KNN-5 (One-vs-Rest) …")
clf    = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5))
clf.fit(emb_tr, df_full3[label_cols].values)

y_pred   = clf.predict(emb_te)
macro_f1 = f1_score(test_df[label_cols].values, y_pred, average="macro")
print(f"\nBaseline macro-F1 = {macro_f1:.3f}")


Кодируем LaBSE-эмбеддинги …


/Users/sabinakalbieva/Documents/add_hse/tools_ad/venv_rubert_sentiment_clean/lib/python3.9/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 11/11 [00:00<00:00, 13.67it/s]


Обучаем KNN-5 (One-vs-Rest) …

Baseline macro-F1 = 0.668


### Увеличение длины отзывов

In [ ]:
import os
import re
import time
from typing import List

import pandas as pd
from tqdm import tqdm
from openai import OpenAI, APIError, APIConnectionError, RateLimitError

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

OPENAI_KEY   = "sk-proj-"
INPUT_PATH  = "train_80_old.xlsx" 
OUTPUT_PATH = "enriched_dataset_old.xlsx" 

LABELS = [
    "Нравится скорость отработки заявок",
    "Нравится качество выполнения заявки",
    "Нравится качество работы сотрудников",
    "Понравилось выполнение заявки",
    "Вопрос решен",
]

SYSTEM = (
    "Ты пишешь короткие (1–2 предложения) дополнения к отзывам жильцов "
    "управляющей компании. Пиши строго на русском. Не повторяй текст "
    "исходного отзыва, не используй кавычки, заголовки, двоеточия или "
    "маркеры, просто добавь новые предложения подряд."
)

client = OpenAI(api_key=OPENAI_KEY)


def make_user_prompt(comment: str, active_labels: List[str]) -> str:
    """Формирует user‑prompt для ChatGPT."""

    if active_labels:
        aspects = "\n".join(f"- {lbl}" for lbl in active_labels)
        task = (
            "Допиши к отзыву 1–2 уникальных предложения, чтобы косвенно "
            "затронуть КАЖДЫЙ аспект из списка ниже. Не упоминай названий "
            "аспектов, не вставляй заголовков, кавычек и двоеточий. Просто "
            "продолжи текст в том же стиле.\n\n"
            f"Аспекты (не показывать в ответе):\n{aspects}"
        )
    else:
        task = (
            "Продолжи отзыв 1–2 предложениями в том же стиле, без шаблонных "
            "благодарностей и без повторения исходного текста."
        )

    return (
        f"Оригинал отзыва (не повторять):\n{comment}\n\n{task}"
    )


def clean_addition(addition: str, comment: str) -> str:
    text = addition.replace("\n", " ").strip()

    text = re.sub(r"[«»\"']", "", text)

    for lbl in LABELS:
        pattern = re.escape(lbl) + r"\s*:"
        text = re.sub(pattern, "", text, flags=re.I)

    text = re.sub(r"^\s*[\-–]\s*", "", text)

    text = text.replace(comment, "")

    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()


def gpt_enrich(row) -> str:
    """Возвращает обогащённую версию отзыва либо исходник при ошибке."""

    comment = row["comment"].strip()
    active  = [lbl for lbl in LABELS if row.get(lbl, 0) == 1]
    prompt  = make_user_prompt(comment, active)

    for attempt in range(3):
        try:
            resp = client.chat.completions.create(
                model="gpt-3.5-turbo-0125",
                temperature=0.7,
                messages=[
                    {"role": "system", "content": SYSTEM},
                    {"role": "user",   "content": prompt},
                ],
                timeout=60,
            )
            raw_add = resp.choices[0].message.content.strip()
            add     = clean_addition(raw_add, comment)
            return f"{comment} {add}".strip()
        except (APIError, APIConnectionError, RateLimitError):
            time.sleep(1)

    return comment 

synthetic5 = pd.read_excel(INPUT_PATH)

tqdm.pandas(desc="Обогащаем отзывы")
synthetic5["comment_enriched"] = synthetic5.progress_apply(gpt_enrich, axis=1)
synthetic5.to_excel(OUTPUT_PATH, index=False)

Обогащаем отзывы: 100%|██████████| 1288/1288 [23:51<00:00,  1.11s/it]


In [ ]:
cols_to_keep = [
    "comment_enriched",
    "Нравится скорость отработки заявок",
    "Нравится качество выполнения заявки",
    "Нравится качество работы сотрудников",
    "Понравилось выполнение заявки",
    "Вопрос решен"
]

synthetic5 = synthetic5[cols_to_keep]

synthetic5 = synthetic5.rename(columns={"comment_enriched": "comment"})

In [45]:
synthetic5.head()

,comment,Нравится скорость отработки заявок,Нравится качество выполнения заявки,Нравится качество работы сотрудников,Понравилось выполнение заявки,Вопрос решен
0,На вопрос ответили. Крепления нельзя повесить ...,0,0,0,0,1
1,спасибо сегодня возвращался домой и запаха не ...,0,0,0,1,0
2,"мастер выполнил работу быстро и чисто, все отл...",0,1,1,0,0
3,оперативно восстановили элетроэнергию. Жильцы ...,1,1,0,0,1
4,"Молодцы, спасибо Благодарю за оперативность, в...",0,0,0,1,0


In [46]:
print(f'Строк: {synthetic5.shape[0]:,}, столбцов: {synthetic5.shape[1]}')

Строк: 1,288, столбцов: 6


In [47]:
print("\nКодируем LaBSE-эмбеддинги …")
labse  = SentenceTransformer("sentence-transformers/LaBSE")
emb_tr = labse.encode(synthetic5["comment"].tolist(), show_progress_bar=True)
emb_te = labse.encode(test_df["comment"].tolist(),  show_progress_bar=True)

print("Обучаем KNN-5 (One-vs-Rest) …")
clf    = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5))
clf.fit(emb_tr, synthetic5[label_cols].values)

y_pred   = clf.predict(emb_te)
macro_f1 = f1_score(test_df[label_cols].values, y_pred, average="macro")
print(f"\nBaseline macro-F1 = {macro_f1:.3f}")


Кодируем LaBSE-эмбеддинги …


/Users/sabinakalbieva/Documents/add_hse/tools_ad/venv_rubert_sentiment_clean/lib/python3.9/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 11/11 [00:00<00:00, 11.59it/s]


Обучаем KNN-5 (One-vs-Rest) …

Baseline macro-F1 = 0.685
